### Experiments (t-tests) were conducted at gas stations with __brand 1 (predicted consumption of brand 1 based on past data)__ and  __brand 2 (consumption at the moment)__

# Data 2021 (seasonal coef)

In [50]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn import datasets
import seaborn as sns

import scipy.stats as stats
from scipy.stats import uniform, binom, norm, kstest, shapiro 

import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')


In [51]:
dfbrand1_2156 = pd.read_sql('''
select discount_card_info, sum(quantity) as goods, count(distinct cheque_id) as cheks
from 
(select * from pet.pet_0321
union all
select * from pet.pet_0421
union all
select * from pet.pet_0521) pet
where discount_card_info is not null
and cheque_type=0
and pet.volume is null
and pet.org_ksss in (159202, 159203, 63604, 104801, 5072, 3247, 5045, 9313, 9312, 9311)
and pet.gds_ksss in (3248159, 3248135, 3248083, 3248079, 3248078, 3248082, 3248081, 3248066, 3248067, 3248065, 3248071, 3248068)
group by discount_card_info
''', connection)

In [52]:
dfbrand1_21789 = pd.read_sql('''
select discount_card_info, sum(quantity) as goods, count(distinct cheque_id) as cheks
from 
(select * from pet.pet_0721
union all
select * from pet.pet_0821
union all 
select * from pet.pet_0921) pet
where discount_card_info is not null
and cheque_type=0
and pet.volume is null
and pet.org_ksss in ('159202', '159203', '63604', '104801', '5072', '3247', '5045', '9313', '9312', '9311')
and pet.gds_ksss in (3248159, 3248135, 3248083, 3248079, 3248078, 3248082, 3248081, 3248066, 3248067, 3248065, 3248071, 3248068)
group by discount_card_info
''', connection)

## Deleting outliers

In [53]:
dfbrand1_2156 = dfbrand1_2156[dfbrand1_2156['goods'] <= np.quantile(dfbrand1_2156['goods'],0.95)]
dfbrand1_21789 = dfbrand1_21789[dfbrand1_21789['goods'] <= np.quantile(dfbrand1_21789['goods'],0.95)]
dfbrand1_2156 = dfbrand1_2156[dfbrand1_2156['cheks'] <= np.quantile(dfbrand1_2156['cheks'],0.95)]
dfbrand1_21789 = dfbrand1_21789[dfbrand1_21789['cheks'] <= np.quantile(dfbrand1_21789['cheks'],0.95)]

## Seasonality in the number of goods and checks

In [54]:
coefgoods=dfbrand1_21789['goods'].sum()/dfbrand1_2156['goods'].sum()
coefcheks=dfbrand1_21789['cheks'].sum()/dfbrand1_2156['cheks'].sum()

# Data 2022 (05-06, 07-08-09)

In [57]:
dfbrand1_2256 = pd.read_sql('''
select distinct discount_card_info, sum(case when
                               pet.gds_ksss in (3248159, 3248135, 3248083, 3248079, 3248078, 3248082, 3248081, 3248066, 3248067, 3248065, 3248071, 3248068)
                               then quantity else 0 end) as goods_brand1,
                           count(distinct case when  
                               pet.gds_ksss in (3248159, 3248135, 3248083, 3248079, 3248078, 3248082, 3248081, 3248066, 3248067, 3248065, 3248071, 3248068)
                               then cheque_id else null end) as cheks_brand1,
                           sum(case when
                               pet.gds_ksss in (3438463, 3438468, 3438462, 3438464, 3438460, 3438459, 3438461, 3438451, 3438450, 3439617, 3438457, 3438456,
3438458, 3438447, 3438446, 3438448, 3438443, 3438442, 3438444, 3438449)
                               then quantity else 0 end) as goods_brand2,
                           count(distinct case when 
                               pet.gds_ksss in (3438463, 3438468, 3438462, 3438464, 3438460, 3438459, 3438461, 3438451, 3438450, 3439617, 3438457, 3438456,
3438458, 3438447, 3438446, 3438448, 3438443, 3438442, 3438444, 3438449)
                               then cheque_id else null end) as cheks_brand2
from
(select * from pet.pet_0322
union all
select * from pet.pet_0422
union all
select * from pet.pet_0522) pet
where discount_card_info is not null 
and cheque_type=0 
and pet.volume is null 
and pet.org_ksss in (159202, 159203, 63604, 104801, 5072, 3247, 5045, 9313, 9312, 9311)
and discount_card_info in
(select distinct discount_card_info
 from 
(select distinct discount_card_info
from 
(select * from pet.pet_0322
union all
select * from pet.pet_0422
union all
select * from pet.pet_0522) pet
where discount_card_info is not null
and cheque_type=0
and pet.volume is null
and pet.org_ksss in (159202, 159203, 63604, 104801, 5072, 3247, 5045, 9313, 9312, 9311)
and pet.gds_ksss in (3248159, 3248135, 3248083, 3248079, 3248078, 3248082, 3248081, 3248066, 3248067, 3248065, 3248071, 3248068, 3438463, 3438468, 3438462, 3438464, 3438460, 3438459, 3438461, 3438451, 3438450, 3439617, 3438457, 3438456,
3438458, 3438447, 3438446, 3438448, 3438443, 3438442, 3438444, 3438449)) a
inner join
(select distinct discount_card_info
from 
(select * from pet.pet_0722
union all
select * from pet.pet_0822
union all 
select * from pet.pet_0922) pet
where discount_card_info is not null
and cheque_type=0
and pet.volume is null
and pet.org_ksss in (159202, 159203, 63604, 104801, 5072, 3247, 5045, 9313, 9312, 9311)
and pet.gds_ksss in (3248159, 3248135, 3248083, 3248079, 3248078, 3248082, 3248081, 3248066, 3248067, 3248065, 3248071, 3248068, 3438463, 3438468, 3438462, 3438464, 3438460, 3438459, 3438461, 3438451, 3438450, 3439617, 3438457, 3438456,
3438458, 3438447, 3438446, 3438448, 3438443, 3438442, 3438444, 3438449)) b
using (discount_card_info) )
group by discount_card_info 
''', connection)

In [58]:
dfbrand2_22789 = pd.read_sql('''
select distinct discount_card_info, sum(case when
                               pet.gds_ksss in (3248159, 3248135, 3248083, 3248079, 3248078, 3248082, 3248081, 3248066, 3248067, 3248065, 3248071, 3248068)
                               then quantity else 0 end) as goods_brand1,
                           count(distinct case when  
                               pet.gds_ksss in (3248159, 3248135, 3248083, 3248079, 3248078, 3248082, 3248081, 3248066, 3248067, 3248065, 3248071, 3248068)
                               then cheque_id else null end) as cheks_brand1,
                           sum(case when
                               pet.gds_ksss in (3438463, 3438468, 3438462, 3438464, 3438460, 3438459, 3438461, 3438451, 3438450, 3439617, 3438457, 3438456,
3438458, 3438447, 3438446, 3438448, 3438443, 3438442, 3438444, 3438449)
                               then quantity else 0 end) as goods1,
                           count(distinct case when 
                               pet.gds_ksss in (3438463, 3438468, 3438462, 3438464, 3438460, 3438459, 3438461, 3438451, 3438450, 3439617, 3438457, 3438456,
3438458, 3438447, 3438446, 3438448, 3438443, 3438442, 3438444, 3438449)
                               then cheque_id else null end) as cheks1
from
(select * from pet.pet_0722
union all
select * from pet.pet_0822
union all
select * from pet.pet_0922) pet
where discount_card_info is not null 
and cheque_type=0 
and pet.volume is null 
and pet.org_ksss in (159202, 159203, 63604, 104801, 5072, 3247, 5045, 9313, 9312, 9311)
and discount_card_info in
(select distinct discount_card_info
 from 
(select distinct discount_card_info
from 
(select * from pet.pet_0322
union all
select * from pet.pet_0422
union all
select * from pet.pet_0522) pet
where discount_card_info is not null
and cheque_type=0
and pet.volume is null
and pet.org_ksss in (159202, 159203, 63604, 104801, 5072, 3247, 5045, 9313, 9312, 9311)
and pet.gds_ksss in (3248159, 3248135, 3248083, 3248079, 3248078, 3248082, 3248081, 3248066, 3248067, 3248065, 3248071, 3248068, 3438463, 3438468, 3438462, 3438464, 3438460, 3438459, 3438461, 3438451, 3438450, 3439617, 3438457, 3438456,
3438458, 3438447, 3438446, 3438448, 3438443, 3438442, 3438444, 3438449)) a
inner join
(select distinct discount_card_info
from 
(select * from pet.pet_0722
union all
select * from pet.pet_0822
union all 
select * from pet.pet_0922) pet
where discount_card_info is not null
and cheque_type=0
and pet.volume is null
and pet.org_ksss in (159202, 159203, 63604, 104801, 5072, 3247, 5045, 9313, 9312, 9311)
and pet.gds_ksss in (3248159, 3248135, 3248083, 3248079, 3248078, 3248082, 3248081, 3248066, 3248067, 3248065, 3248071, 3248068, 3438463, 3438468, 3438462, 3438464, 3438460, 3438459, 3438461, 3438451, 3438450, 3439617, 3438457, 3438456,
3438458, 3438447, 3438446, 3438448, 3438443, 3438442, 3438444, 3438449)) b
using (discount_card_info) )
group by discount_card_info 
''', connection)

## Deleting outliers 

In [59]:
dfbrand1_2256 = dfbrand1_2256[dfbrand1_2256['goods_brand1'] <= np.quantile(dfbrand1_2256['goods_brand1'],0.95)]
dfbrand2_22789 = dfbrand2_22789[dfbrand2_22789['goods1'] <= np.quantile(dfbrand2_22789['goods1'],0.95)]
dfbrand1_2256 = dfbrand1_2256[dfbrand1_2256['cheks_brand1'] <= np.quantile(dfbrand1_2256['cheks_brand1'],0.95)]
dfbrand2_22789 = dfbrand2_22789[dfbrand2_22789['cheks1'] <= np.quantile(dfbrand2_22789['cheks1'],0.95)]

## Adding coef to brand1 dataset (suggesting brand1 07-08-09) and deleting data without coef

In [62]:
dfbrand1_2256['goods1'] = dfbrand1_2256['goods_brand1']*coefgoods
dfbrand1_2256['cheks1'] = dfbrand1_2256['cheks_brand1']*coefcheks

In [63]:
dfbrand1_2256 = dfbrand1_2256.drop(['goods_brand1','cheks_brand1'], axis = 1)

## Adding labels

In [64]:
dfbrand1_2256['label']=0
dfbrand2_22789['label']=1
data22 = pd.concat([dfbrand1_2256,dfbrand2_22789], axis = 0, ignore_index = True)

# Comparing brand1 07-08-09 (predicted) and brand2 07-08-09 (real)

In [ ]:
# sns.set(rc={'figure.figsize':(11.7,8.27)})
groups = sns.histplot( data = data22,
                    x = 'goods1',
                    hue = 'label',
                    palette = ['r', 'b'],
                    alpha=0.5,
                    kde=True)
plt.show()

In [1]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
groups = sns.histplot( data = data22,
                    x = 'cheks1',
                    hue = 'label',
                    palette = ['r', 'b'],
                    alpha=0.5,
                    kde=True)
plt.show()

## t-test based on goods

In [69]:
kstest(dfbrand1_2256['goods1'], 'norm')

KstestResult(statistic=0.7714658600640135, pvalue=0.0)

In [70]:
kstest(dfbrand2_22789['goods1'], 'norm')

KstestResult(statistic=0.8413447460685429, pvalue=0.0)

In [71]:
res = sm.stats.ttest_ind(dfbrand1_2256['goods1'], dfbrand2_22789['goods1'])
print("Two Sample t-test")
print("t =", res[0])
print("p-value =", res[1])
print("df = ", res[2])

if res[1] > 0.05:
    print('There is a statistical significance. There is a difference between products sold in 2 expirements')
else: 
    print('There is no statistical significance. There is no difference between products sold in 2 expirements')

Two Sample t-test
t = -5.827168052505521
p-value = 6.161048001147867e-09
df =  3413.0


## t-test based on cheks

In [72]:
kstest(dfbrand1_2256['cheks1'], 'norm')

KstestResult(statistic=0.7723212343502035, pvalue=0.0)

In [73]:
kstest(dfbrand2_22789['cheks1'], 'norm')

KstestResult(statistic=0.8413447460685429, pvalue=0.0)

In [74]:
res = sm.stats.ttest_ind(dfbrand1_2256['cheks1'], dfbrand2_22789['cheks1'])
print("Two Sample t-test")
print("t =", res[0])
print("p-value =", res[1])
print("df = ", res[2])

if res[1] > 0.05:
    print('There is a statistical significance. There is a difference between average checks in 2 expirements')
else: 
    print('There is no statistical significance. There is no difference between average checks in 2 expirements')

Two Sample t-test
t = -6.243432169216756
p-value = 4.807604889083429e-10
df =  3413.0
